In [ ]:
import openai
from dotenv import dotenv_values
from pathlib import Path
import json
import json_stream


In [ ]:
CONFIG = dotenv_values(".env")
openai.api_key = CONFIG["OPENAI_API_KEY"]
data_fn = Path("data.json")

In [ ]:
def get_sample():
    with open(data_fn) as f:
        for meta in json_stream.load(f):
            meta = json_stream.to_standard_types(meta)
            yield json.dumps(meta, ensure_ascii=False)

def get_completion(user, system=None, assistant=None, model="gpt-3.5-turbo-16k"):
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": user})
    if assistant:
        messages.append({"role": "assistant", "content": assistant})

    response = openai.Completion.create(
        model=model,
        messages=messages,
        temperature=0.1,  # the lower, the better?
    )
    return response.choices[0].message["content"]

In [ ]:
basic_prompt = """
你的任务是从湖北省政务文件中提取关键信息，整理出问题及对应的答案，并使用jsonl的输出格式，你可以按照以下步骤进行：
1. 编写问题和回答对：从湖北省政务文件中提取关键信息，编写15-20对的问题和相应详细答案，确保回答能够清晰解释相关政务事项。
2. 输出为jsonl格式： 将每个问题和回答对整理成一个json对象，并按照jsonl格式输出到文件中。使用"input"作为键来表示问题，使用"output"作为键来表示回答，不同问题和回答之间使用\n分隔。例如：
{"input": 问题1, “output”: 回答1}\n{"input": 问题2, “output”: 回答2}
湖北省政务文件是json格式，内容如下：
"""

# Case1: basic prompt as system prompt

In [ ]:
sample = next(iter(get_sample()))
get_completion(sample, system=basic_prompt)

# Case2: basic prompt as user prompt

In [ ]:
user_prompt = basic_prompt + "{document}"
user_prompt.format(document=sample)
get_completion(user_prompt)

# Case3: basic prompt as system/user prompt with fewshot

In [ ]:
get_completion(user_prompt, assistant="# TODO: fewshot example")
get_completion(sample, system=basic_prompt, assistant="# TODO: fewshot example")